In [50]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

os.environ['OPENAI_API_KEY'] ="KEY"

model = ChatOpenAI(model="gpt-4-turbo-preview")

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})



'Why did the ice cream truck break down?\n\nBecause of the rocky road!'

RAG Search

In [23]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser



'Bears like to eat honey.'

In [24]:
chain.invoke("what is the date today?")

"I'm sorry, but I cannot provide the current date as my responses are based on the information provided in the given context, which does not include any information about the current date."

In [21]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

FEW-SHOT PROMPT

In [44]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [60]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [52]:
examples=[
    {
      "role": "system",
      "content": "I want to generate an access control policy as a json file. In this policy, U2 uses Alexa and U1 uses Smartthings. U2 sends a command to U1 to request the access to the smart device. The request looks like this \"U2 wants to turn on the switch in the living room in U1's account at 08:00, Day. U1 allows.\" and the json file looks like this \n\n{\n    \"name\": \"shadowConn\",\n    \"trigger\": {\n      \"src\": {\n        \"user\": \"U1\",\n        \"platform\": [\"Smartthings\"],\n        \"device\": \"switch\"\n      },\n      \"dst\": {\n        \"user\": \"U2\",\n        \"platform\": [\"Alexa\"],\n        \"device\": \"switch\"\n      }\n    },\n    \"condition\": {\n      \"location\": [\"living room\"],\n      \"time\": [\"08:00\", \"day\"]\n    },\n    \"decision\": {\n      \"requester\": \"U2\",\n      \"approver\": \"U1\",\n      \"outcome\": [\"accept\"]\n    },\n    \"actions\": [\n      {\n        \"if\": {\n          \"decision\": \"accept\",\n          \"equals\": {\n            \"left\": {\n              \"device\": {\n                \"deviceId\": \"\",\n                \"component\": \"main\",\n                \"capability\": \"switch\",\n                \"attribute\": \"switch\"\n              }\n            },\n            \"right\": {\n              \"string\": \"on\"\n            }\n          },\n          \"then\": [\n            {\n              \"command\": {\n                \"devices\": [\"\"],\n                \"commands\": [\n                  {\n                    \"component\": \"main\",\n                    \"capability\": \"switch\",\n                    \"command\": \"on\"\n                  }\n                ]\n              }\n            }\n          ],\n          \"else\": [\n            {\n              \"command\": {\n                \"devices\": [\"\"],\n                \"commands\": [\n                  {\n                    \"component\": \"main\",\n                    \"capability\": \"switch\",\n                    \"command\": \"off\"\n                  }\n                ]\n              }\n            }\n          ]\n        }\n      }\n    ]\n  }\n\n. I want the result with the correct JSON format. No extra explaination"
    },
    {
      "role": "user",
      "content": "U2 wants to turn on the switch in the living room in U1's account at 22:00, Night. U1 rejects."
    },
    {
      "role": "assistant",
      "content": "Based on the new request where U2 wants to turn on the switch in the living room in U1's account at 22:00, Night, and U1 rejects the request, the JSON file reflecting this scenario would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"switch\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"switch\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"living room\"],\n    \"time\": [\"22:00\", \"night\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"reject\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"switch\",\n              \"attribute\": \"switch\"\n            }\n          },\n          \"right\": {\n            \"string\": \"on\"\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"switch\",\n                  \"command\": \"on\"\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"switch\",\n                  \"command\": \"off\"\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nIn this updated JSON file, the `\"condition\"` section now reflects the new requested time of `\"22:00\", \"night\"`, and the `\"decision\"` section now indicates that the outcome is `\"reject\"`. The actions remain structured to handle both acceptance and rejection scenarios, but given the decision, the relevant action would be to keep the switch off, as indicated in the `\"else\"` part of the `\"if\"` statement."
    },
    {
      "role": "user",
      "content": "U2 wants to control the smart window blinds in the bedroom in U1's account at 11:00, Day. U1 rejects.\n"
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to control the smart window blinds in the bedroom in U1's account at 11:00, Day, and U1 rejects the request, the JSON file would be structured as follows:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"window blinds\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"window blinds\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"bedroom\"],\n    \"time\": [\"11:00\", \"day\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"reject\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"windowBlinds\",\n              \"attribute\": \"windowBlinds\"\n            }\n          },\n          \"right\": {\n            \"string\": \"open\" // Assuming the desired action was to open the blinds\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"windowBlinds\",\n                  \"command\": \"open\"\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"windowBlinds\",\n                  \"command\": \"close\" // Assuming the default or rejection state is to close the blinds\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nThis JSON file reflects the specific request involving the smart window blinds in the bedroom, with the condition set for 11:00 during the day. Given that U1 rejects the request, the decision is marked as `\"reject\"`, and the actions are structured to handle both acceptance and rejection scenarios. In this case, the `\"else\"` part of the `\"if\"` statement indicates the action to be taken upon rejection, which is assumed to be closing the blinds. Note that the specific commands for controlling window blinds (e.g., `\"open\"`, `\"close\"`) might need to be adjusted based on the actual capabilities of the smart device and the platform's API."
    },
    {
      "role": "user",
      "content": "U2 wants to adjust the smart lighting system in the living room in U1's account at 20:00, Night. U1 allows."
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to adjust the smart lighting system in the living room in U1's account at 20:00, Night, and U1 allows the request, the JSON file would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart lighting\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart lighting\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"living room\"],\n    \"time\": [\"20:00\", \"night\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"accept\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartLighting\",\n              \"attribute\": \"smartLighting\"\n            }\n          },\n          \"right\": {\n            \"string\": \"adjust\" // Assuming 'adjust' involves changing brightness or color\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartLighting\",\n                  \"command\": \"setLevel\", // Example command for adjusting brightness\n                  \"arguments\": [\"75\"] // Example argument, assuming 75% brightness\n                },\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartLighting\",\n                  \"command\": \"setColor\", // Additional example command for adjusting color\n                  \"arguments\": [\"warm\"] // Example argument, assuming warm color\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartLighting\",\n                  \"command\": \"off\" // Assuming the default or rejection state is to turn off\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nThis JSON structure reflects the request to adjust the smart lighting system in the living room at 20:00 during the night, with U1 allowing the request. The decision is marked as `\"accept\"`, and the actions are structured to handle the accepted scenario. The `\"then\"` part of the `\"if\"` statement includes example commands for adjusting the brightness and color of the smart lighting system, assuming these capabilities exist. The specific commands (`\"setLevel\"`, `\"setColor\"`, and their arguments) might need to be adjusted based on the actual capabilities of the smart lighting system and the platform's API."
    },
    {
      "role": "user",
      "content": "U2 wants to turn off the smart air purifier in the living room in U1's account at 12:00, Day. U1 rejects."
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to turn off the smart air purifier in the living room in U1's account at 12:00, Day, and U1 rejects the request, the JSON file would be structured as follows:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart air purifier\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart air purifier\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"living room\"],\n    \"time\": [\"12:00\", \"day\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"reject\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartAirPurifier\",\n              \"attribute\": \"smartAirPurifier\"\n            }\n          },\n          \"right\": {\n            \"string\": \"off\"\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartAirPurifier\",\n                  \"command\": \"off\"\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartAirPurifier\",\n                  \"command\": \"on\" // Assuming the default or rejection state is to keep the purifier on\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nIn this JSON structure, the request to turn off the smart air purifier in the living room at 12:00 during the day is detailed, with U1 rejecting the request. The decision is marked as `\"reject\"`, and the actions are structured to handle both acceptance and rejection scenarios. Given the decision to reject, the relevant action would be to keep the air purifier on, as indicated in the `\"else\"` part of the `\"if\"` statement. The specific commands for controlling the smart air purifier (`\"on\"` or `\"off\"`) are included, assuming these are the correct commands supported by the device and platform API."
    },
    {
      "role": "user",
      "content": "U2 wants to use the smart oven in the kitchen in U1's account at 07:30, Day. U1 allows.\n"
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to use the smart oven in the kitchen in U1's account at 07:30, Day, and U1 allows the request, the JSON file would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart oven\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart oven\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"kitchen\"],\n    \"time\": [\"07:30\", \"day\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"accept\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartOven\",\n              \"attribute\": \"smartOven\"\n            }\n          },\n          \"right\": {\n            \"string\": \"on\" // Assuming the desired action is to turn the oven on or start preheating\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartOven\",\n                  \"command\": \"preheat\", // Example command for starting the oven\n                  \"arguments\": [\"350\"] // Example argument, assuming 350 degrees Fahrenheit for preheating\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartOven\",\n                  \"command\": \"off\" // Assuming the default or rejection state is to turn the oven off\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nThis JSON structure reflects the request to use the smart oven in the kitchen at 07:30 during the day, with U1 allowing the request. The decision is marked as `\"accept\"`, and the actions are structured to handle the accepted scenario. The `\"then\"` part of the `\"if\"` statement includes an example command for starting or preheating the oven, assuming these are the correct capabilities of the smart oven and supported by the platform API. The specific command (`\"preheat\"`) and its argument (`\"350\"`) are examples and might need to be adjusted based on the actual capabilities of the smart oven and the platform's API."
    },
    {
      "role": "user",
      "content": "U2 wants to adjust the smart thermostat in the kitchen in U1's account at 22:00, Night. U1 rejects."
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to adjust the smart thermostat in the kitchen in U1's account at 22:00, Night, and U1 rejects the request, the JSON file would be structured as follows:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart thermostat\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart thermostat\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"kitchen\"],\n    \"time\": [\"22:00\", \"night\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"reject\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartThermostat\",\n              \"attribute\": \"smartThermostat\"\n            }\n          },\n          \"right\": {\n            \"string\": \"adjust\" // Assuming 'adjust' involves changing temperature\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartThermostat\",\n                  \"command\": \"setTemperature\", // Example command for adjusting temperature\n                  \"arguments\": [\"68\"] // Example argument, assuming 68 degrees Fahrenheit\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartThermostat\",\n                  \"command\": \"maintainTemperature\", // Assuming the default or rejection state is to maintain current temperature\n                  \"arguments\": [\"current\"] // Placeholder argument, indicating to maintain the current setting\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nIn this JSON structure, the request to adjust the smart thermostat in the kitchen at 22:00 during the night is detailed, with U1 rejecting the request. The decision is marked as `\"reject\"`, and the actions are structured to handle both acceptance and rejection scenarios. Given the decision to reject, the relevant action would be to maintain the current temperature setting, as indicated in the `\"else\"` part of the `\"if\"` statement. The specific commands for controlling the smart thermostat (`\"setTemperature\"`, `\"maintainTemperature\"`, and their arguments) are examples and might need to be adjusted based on the actual capabilities of the smart thermostat and the platform's API."
    },
    {
      "role": "user",
      "content": "U2 wants to control the smart TV in the bedroom in U1's account at 21:30, Night. U1 allows."
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to control the smart TV in the bedroom in U1's account at 21:30, Night, and U1 allows the request, the JSON configuration would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart TV\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart TV\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"bedroom\"],\n    \"time\": [\"21:30\", \"night\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"accept\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartTV\",\n              \"attribute\": \"smartTV\"\n            }\n          },\n          \"right\": {\n            \"string\": \"on\" // Assuming the desired action is to turn the TV on\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartTV\",\n                  \"command\": \"on\"\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartTV\",\n                  \"command\": \"off\" // Assuming the default or rejection state is to turn the TV off\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nThis JSON structure reflects the request to control the smart TV in the bedroom at 21:30 during the night, with U1 allowing the request. The decision is marked as `\"accept\"`, and the actions are structured to handle the accepted scenario. The `\"then\"` part of the `\"if\"` statement includes the command to turn the smart TV on, as this is the assumed desired action based on the request. The specific command (`\"on\"`) is included, assuming this is the correct command supported by the smart TV and the platform API."
    },
    {
      "role": "user",
      "content": "U2 wants to turn on the smart alarm clock in the bedroom in U1's account at 06:00, Day. U1 rejects."
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to turn on the smart alarm clock in the bedroom in U1's account at 06:00, Day, and U1 rejects the request, the JSON configuration would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart alarm clock\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart alarm clock\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"bedroom\"],\n    \"time\": [\"06:00\", \"day\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"reject\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartAlarmClock\",\n              \"attribute\": \"smartAlarmClock\"\n            }\n          },\n          \"right\": {\n            \"string\": \"on\" // Assuming the desired action is to set or turn on the alarm\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartAlarmClock\",\n                  \"command\": \"setAlarm\",\n                  \"arguments\": [\"06:00\"] // Example argument, assuming setting the alarm for 6:00 AM\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartAlarmClock\",\n                  \"command\": \"off\" // Assuming the default or rejection state is to keep the alarm off\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nIn this JSON structure, the request to turn on the smart alarm clock in the bedroom at 06:00 during the day is detailed, with U1 rejecting the request. The decision is marked as `\"reject\"`, and the actions are structured to handle both acceptance and rejection scenarios. Given the decision to reject, the relevant action would be to keep the alarm clock off, as indicated in the `\"else\"` part of the `\"if\"` statement. The specific commands for controlling the smart alarm clock (`\"setAlarm\"`, `\"off\"`, and their arguments) are examples and might need to be adjusted based on the actual capabilities of the smart alarm clock and the platform's API."
    },
    {
      "role": "user",
      "content": "U2 wants to engage the smart floor lamp in the living room in U1's account at 19:00, Evening. U1 allows."
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to engage the smart floor lamp in the living room in U1's account at 19:00, Evening, and U1 allows the request, the JSON configuration would look as follows:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart floor lamp\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart floor lamp\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"living room\"],\n    \"time\": [\"19:00\", \"evening\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"accept\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartFloorLamp\",\n              \"attribute\": \"smartFloorLamp\"\n            }\n          },\n          \"right\": {\n            \"string\": \"on\" // Assuming the desired action is to turn the lamp on\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartFloorLamp\",\n                  \"command\": \"on\"\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartFloorLamp\",\n                  \"command\": \"off\" // Assuming the default or rejection state is to turn the lamp off\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nThis JSON structure reflects the request to engage the smart floor lamp in the living room at 19:00 during the evening, with U1 allowing the request. The decision is marked as `\"accept\"`, and the actions are structured to handle the accepted scenario. The `\"then\"` part of the `\"if\"` statement includes the command to turn the smart floor lamp on, as this is the assumed desired action based on the request. The specific command (`\"on\"`) is included, assuming this is the correct command supported by the smart floor lamp and the platform API."
    },
    {
      "role": "user",
      "content": "U2 wants to operate the smart wall plug in the living room in U1's account at 15:00, Day. U1 rejects."
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to operate the smart wall plug in the living room in U1's account at 15:00, Day, and U1 rejects the request, the JSON configuration would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart wall plug\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart wall plug\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"living room\"],\n    \"time\": [\"15:00\", \"day\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"reject\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartWallPlug\",\n              \"attribute\": \"smartWallPlug\"\n            }\n          },\n          \"right\": {\n            \"string\": \"on\" // Assuming the desired action is to turn the plug on\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartWallPlug\",\n                  \"command\": \"on\"\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartWallPlug\",\n                  \"command\": \"off\" // Assuming the default or rejection state is to keep the plug off\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nIn this JSON structure, the request to operate the smart wall plug in the living room at 15:00 during the day is detailed, with U1 rejecting the request. The decision is marked as `\"reject\"`, and the actions are structured to handle both acceptance and rejection scenarios. Given the decision to reject, the relevant action would be to keep the smart wall plug off, as indicated in the `\"else\"` part of the `\"if\"` statement. The specific commands for controlling the smart wall plug (`\"on\"` or `\"off\"`) are included, assuming these are the correct commands supported by the smart wall plug and the platform API."
    },
    {
      "role": "user",
      "content": "U2 wants to activate the smart garage door opener in the garage room in U1's account at 20:30, Night. U1 allows. I want the result with the correct JSON format. No extra information or explanation"
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to activate the smart garage door opener in the garage room in U1's account at 20:30, Night, and U1 allows the request, the JSON configuration would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart garage door opener\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart garage door opener\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"garage room\"],\n    \"time\": [\"20:30\", \"night\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"accept\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartGarageDoorOpener\",\n              \"attribute\": \"smartGarageDoorOpener\"\n            }\n          },\n          \"right\": {\n            \"string\": \"activate\" // Assuming the desired action is to open the garage door\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartGarageDoorOpener\",\n                  \"command\": \"open\" // Command to open the garage door\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartGarageDoorOpener\",\n                  \"command\": \"close\" // Assuming the default or rejection state is to keep the garage door closed\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nThis JSON structure reflects the request to activate (open) the smart garage door opener in the garage room at 20:30 during the night, with U1 allowing the request. The decision is marked as `\"accept\"`, and the actions are structured to handle the accepted scenario. The `\"then\"` part of the `\"if\"` statement includes the command to open the smart garage door opener, as this is the assumed desired action based on the request. The specific command (`\"open\"`) is included, assuming this is the correct command supported by the smart garage door opener and the platform API."
    },
    {
      "role": "user",
      "content": "U2 wants to use the smart irrigation system in the garden through U1's account at 14:30, Day. U1 rejects. I want the result with the correct JSON format. No extra information or explanation"
    },
    {
      "role": "assistant",
      "content": "For the scenario where U2 wants to use the smart irrigation system in the garden through U1's account at 14:30, Day, and U1 rejects the request, the JSON configuration would look like this:\n\n```json\n{\n  \"name\": \"shadowConn\",\n  \"trigger\": {\n    \"src\": {\n      \"user\": \"U1\",\n      \"platform\": [\"Smartthings\"],\n      \"device\": \"smart irrigation system\"\n    },\n    \"dst\": {\n      \"user\": \"U2\",\n      \"platform\": [\"Alexa\"],\n      \"device\": \"smart irrigation system\"\n    }\n  },\n  \"condition\": {\n    \"location\": [\"garden\"],\n    \"time\": [\"14:30\", \"day\"]\n  },\n  \"decision\": {\n    \"requester\": \"U2\",\n    \"approver\": \"U1\",\n    \"outcome\": [\"reject\"]\n  },\n  \"actions\": [\n    {\n      \"if\": {\n        \"decision\": \"accept\",\n        \"equals\": {\n          \"left\": {\n            \"device\": {\n              \"deviceId\": \"\",\n              \"component\": \"main\",\n              \"capability\": \"smartIrrigationSystem\",\n              \"attribute\": \"smartIrrigationSystem\"\n            }\n          },\n          \"right\": {\n            \"string\": \"activate\" // Assuming the desired action is to start watering\n          }\n        },\n        \"then\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartIrrigationSystem\",\n                  \"command\": \"startWatering\" // Example command to start the irrigation system\n                }\n              ]\n            }\n          }\n        ],\n        \"else\": [\n          {\n            \"command\": {\n              \"devices\": [\"\"],\n              \"commands\": [\n                {\n                  \"component\": \"main\",\n                  \"capability\": \"smartIrrigationSystem\",\n                  \"command\": \"stopWatering\" // Assuming the default or rejection state is to stop or not start watering\n                }\n              ]\n            }\n          }\n        ]\n      }\n    }\n  ]\n}\n```\n\nIn this JSON structure, the request to use the smart irrigation system in the garden at 14:30 during the day is detailed, with U1 rejecting the request. The decision is marked as `\"reject\"`, and the actions are structured to handle both acceptance and rejection scenarios. Given the decision to reject, the relevant action would be to not start or to stop watering, as indicated in the `\"else\"` part of the `\"if\"` statement. The specific commands for controlling the smart irrigation system (`\"startWatering\"`, `\"stopWatering\"`) are examples and might need to be adjusted based on the actual capabilities of the smart irrigation system and the platform's API."
    }
  ]

In [105]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = "KEY"


prompt = ChatPromptTemplate.from_template(
    """
    I want to generate an access control policy as a json file from user request. The json file looks like this:
        {{
        "trigger": {{
            "src": {{
                "user1": "U1",
                "platform": "Smartthings",
            }},
            "dst": {{
                "user": "U2",
                "platform": "Alexa",
            }},
            "device_type": ""
        }},
        "condition": {{
            "location": [""],
            "time": ["00:00", ["day", "night"]]
        }},
        "decision": {{
            "requester": "",
            "approver": "",
            "outcome": ["accept", "decline"]
        }},
        "actions": [
            {{
                "if": {{
                    "decision": "accept",
                    "equals": {{
                        "left": {{
                            "device": {{
                                "deviceId1": "",
                                "component": "main",
                                "capability": "switch",
                                "attribute": "switch"
                            }}
                        }},
                        "right": {{
                            "string": "on"
                        }}
                    }},
                    "then": [
                        {{
                            "command": {{
                                "devicesId2": "",
                                "commands": [
                                    {{
                                        "component": "main",
                                        "capability": "switch",
                                        "command": "on"
                                    }}
                                ]
                            }}
                        }}
                    ],
                    "else": [
                        {{
                            "command": {{
                                "devicesId2": "",
                                "commands": [
                                    {{
                                        "component": "main",
                                        "capability": "switch",
                                        "command": "off"
                                    }}
                                ]
                            }}
                        }}
                    ]
                }}
            }}
        ]
        }}

    Use this {request} to create the JSON file. No extra explanation needed. User needs to specify the time.
    The device type is the device that is requested. 
    Requester is the user who asking for the permit, and receiver is the user who is asked. 
    Do not change the decision value in the if statement and do not skip the action part. 
    If there is missing information, keep asking user to input the missing information. The outcome of the decision will be accept or decline.
    """
)

output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4-turbo-preview")

chain = (
    {"request": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

In [106]:
result = chain.invoke("U2 wants to use the U1's smart light")

print(result)

To generate an access control policy as a JSON file based on the user request described, we'll follow a step-by-step approach. Please provide the missing information as we go through each step. Let's start with the information provided and what's missing:

1. **Trigger Section:**
   - **src (source):** We know the source user is `U1` and the platform is `Smartthings`.
   - **dst (destination):** The destination user is `U2` and the platform is `Alexa`.
   - **device_type:** Since U2 wants to use U1's smart light, the device type is "smart light".

2. **Condition Section:**
   - You mentioned that the user needs to specify the time. Please provide the **time** during which this policy should be active. Also, specify the **location** if applicable.

3. **Decision Section:**
   - **requester:** Since U2 is the one who wants to use the device, U2 is the requester.
   - **approver:** U1, being the owner of the device, would be the approver.
   - The **outcome** is set to be either "accept" 

In [107]:
result = chain.invoke("""
- The device type is "smart_light".
- The location is "living room".
- The deviceId1 is "Light123". The deviceId2  is "Light123"
- The decision is accept.
- Time is 8:00 am
""")

print(result)

```json
{
    "trigger": {
        "src": {
            "user1": "U1",
            "platform": "Smartthings"
        },
        "dst": {
            "user": "U2",
            "platform": "Alexa"
        },
        "device_type": "smart_light"
    },
    "condition": {
        "location": ["living room"],
        "time": ["08:00", ["day"]]
    },
    "decision": {
        "requester": "U1",
        "approver": "U2",
        "outcome": "accept"
    },
    "actions": [
        {
            "if": {
                "decision": "accept",
                "equals": {
                    "left": {
                        "device": {
                            "deviceId1": "Light123",
                            "component": "main",
                            "capability": "switch",
                            "attribute": "switch"
                        }
                    },
                    "right": {
                        "string": "on"
                    }
                },
   

In [96]:
result = chain.invoke("Missing requester and approver")

print(result)

To generate the access control policy as specified, I need you to provide the missing information:

1. The `device_type` the policy applies to.
2. The `location(s)` this policy applies to.
3. The `requester`'s identity.
4. The `approver`'s identity.
5. The specific `time` range or conditions this policy should consider.
6. The `deviceId1` for the condition check.
7. The `devicesId2` for the action to be taken.

Please provide the missing information to proceed with creating the JSON file.
